In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import plotly
plotly.tools.set_credentials_file(username='quoniammm', api_key='IF7kV6idFRdoo7LdgGRp')
import plotly.plotly as py
import plotly.graph_objs as go
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

import nltk
# ============Pytorch=============
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
# ============Keras===============
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
# ============gensim==============
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.models.keyedvectors import KeyedVectors

%matplotlib inline

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')

Using TensorFlow backend.
/home/quoniammm/anaconda3/envs/py3Tfgpu/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning:

compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6



In [2]:
# 加载 word2vec
# word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [3]:
label_encoder = LabelEncoder()
y = np.array(label_encoder.fit_transform(train.author))

xtrain, xvalid, ytrain, yvalid = train_test_split(
    train.text, y, 
    random_state=42, 
    test_size=0.2, 
    shuffle=True
)
tokenizer = Tokenizer(num_words=20000,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'', lower=True)
tokenizer.fit_on_texts(train.text)
seq_train = tokenizer.texts_to_sequences(xtrain)
seq_valid = tokenizer.texts_to_sequences(xvalid)
x_train = pad_sequences(seq_train, maxlen=256)
x_val = pad_sequences(seq_valid, maxlen=256)
word_index = tokenizer.word_index
len(word_index)

25095

In [4]:
# embedding_matrix = np.zeros((20000, 300))
# for word, i in word_index.items():
#     if i>= 20000:
#         continue
#     try:
#         embedding_vector = word_vectors[word]
#         embedding_matrix[i] = embedding_vector
#     except KeyError:
#         embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),300)

In [5]:
# np.save("em.npy", embedding_matrix)
embedding_matrix = np.load("em.npy")

In [6]:
#TextCNN static 实现
class textCNN(nn.Module):
    def __init__(self, num_tokens, embedded_size, kernel_sizes, kernel_num, dropout_value, class_num):
        super(textCNN, self).__init__()
        self.embed = nn.Embedding(num_tokens, embedded_size)
        self.embed.weight.data.copy_(torch.FloatTensor(embedding_matrix))
        # (3, 4, 5)
        self.convs = nn.ModuleList([nn.Conv2d(1, kernel_num, (K, embedded_size)) for K in kernel_sizes])
        self.dropout = nn.Dropout(dropout_value)
        self.fc = nn.Linear(len(kernel_sizes)*kernel_num ,class_num)
    
    def forward(self, x):
        # (16, 861) (16 861 256)
        x = self.embed(x)
        # (16 1 861 256)
        x =  x.unsqueeze(1)
        # 一次卷积结果：（16 20 dynamic 1)
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs]
        # (16 20 1 1)
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]
        # (16 60)
        x = torch.cat(x, 1)
        # (16, 60)
        x = self.dropout(x)
        # (16, 3)
        logit = self.fc(x)
        #print(logit.size())
        return F.softmax(logit, dim=1), F.log_softmax(logit, dim=1)

In [7]:
lr = 1e-4

model = textCNN(20000, 300, (3,4,5), 100, 0.5, 3)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.NLLLoss()
model.cuda()

textCNN(
  (embed): Embedding(20000, 300)
  (convs): ModuleList(
    (0): Conv2d (1, 100, kernel_size=(3, 300), stride=(1, 1))
    (1): Conv2d (1, 100, kernel_size=(4, 300), stride=(1, 1))
    (2): Conv2d (1, 100, kernel_size=(5, 300), stride=(1, 1))
  )
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=300, out_features=3)
)

In [8]:
epochs = 30
train_acc = []
valid_acc = []
batch_size=256
# 训练模型
for epoch in range(epochs):
    vx = Variable(torch.LongTensor(x_train.astype(int))).cuda()
    vy = Variable(torch.LongTensor(ytrain)).cuda()
    vx_valid = Variable(torch.LongTensor(x_val.astype(int))).cuda()
    vy_valid = Variable(torch.LongTensor(yvalid)).cuda()
    optimizer.zero_grad()
    for i in range(0, len(xtrain), batch_size):
        if i + batch_size > len(xtrain):
            vx_batch = vx[-(batch_size+1):-1]
            vy_batch = vy[-(batch_size+1):-1]
        else:
            vx_batch = vx[i:i+batch_size]
            vy_batch = vy[i:i+batch_size]
            
        
        shuffle = np.random.randint(0, 1958, batch_size)
        vx_valid_batch = vx_valid.index_select(0, Variable(torch.LongTensor(shuffle)).cuda())
        vy_valid_batch = vy_valid.index_select(0, Variable(torch.LongTensor(shuffle)).cuda())
        
        results, outputs = model(vx_batch)
        results_v, outputs_v = model(vx_valid_batch)
        
        loss = criterion(outputs, vy_batch)
        loss_v = criterion(outputs_v, vy_valid_batch)
        loss.backward()
        optimizer.step()
        
        _, res = results.data.max(1)
        _, res_v = results_v.data.max(1)
        
        if not i % 3200:
            print("after {}% training loss={}, valid loss={}, at {} epoch".format(round((float(i) / len(xtrain))*100), loss.data[0], loss_v.data[0], epoch))
            train_acc.append((torch.sum(res == vy_batch.data) + 0.0) / batch_size)
            valid_acc.append((torch.sum(res_v == vy_valid_batch.data) + 0.0) / batch_size)            

after 0% training loss=1.1029483079910278, valid loss=1.1217938661575317, at 0 epoch
after 41% training loss=1.0922152996063232, valid loss=1.0675848722457886, at 0 epoch
after 82% training loss=1.0355325937271118, valid loss=1.0486035346984863, at 0 epoch
after 0% training loss=1.1331849098205566, valid loss=1.1343662738800049, at 1 epoch
after 41% training loss=1.0809779167175293, valid loss=1.0433579683303833, at 1 epoch
after 82% training loss=1.1539726257324219, valid loss=1.2519454956054688, at 1 epoch
after 0% training loss=0.8666999936103821, valid loss=0.9369055032730103, at 2 epoch
after 41% training loss=0.9571783542633057, valid loss=0.93276447057724, at 2 epoch
after 82% training loss=0.9741988182067871, valid loss=0.9747200608253479, at 2 epoch
after 0% training loss=0.8262263536453247, valid loss=0.9813982844352722, at 3 epoch
after 41% training loss=0.827739417552948, valid loss=0.8651456832885742, at 3 epoch
after 82% training loss=0.9473941922187805, valid loss=0.9838

In [9]:
N = len(train_acc)
print(N)
random_x = np.linspace(0, 1, N)
random_y0 = np.array(train_acc)
random_y1 = np.array(valid_acc)

# Create traces
trace0 = go.Scatter(
    x = random_x,
    y = random_y0,
    mode = 'lines',
    name = 'train_acc'
)
trace1 = go.Scatter(
    x = random_x,
    y = random_y1,
    mode = 'lines',
    name = 'valid_acc'
)

data = [trace0, trace1]

py.iplot(data, filename='line-mode')

90


In [10]:
test_data = tokenizer.texts_to_sequences(test.text)
X_t_pad = pad_sequences(test_data, maxlen=256)
tt = Variable(torch.LongTensor(X_t_pad.astype(int)))

In [11]:
model.cpu()
model.eval()

textCNN(
  (embed): Embedding(20000, 300)
  (convs): ModuleList(
    (0): Conv2d (1, 100, kernel_size=(3, 300), stride=(1, 1))
    (1): Conv2d (1, 100, kernel_size=(4, 300), stride=(1, 1))
    (2): Conv2d (1, 100, kernel_size=(5, 300), stride=(1, 1))
  )
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=300, out_features=3)
)

In [12]:
for i in range(0, len(tt), 16):
    if i + 16 > len(tt):
        batch = tt[i:]
    else:
        batch = tt[i:i+16]
        
    r, _ = model(batch)
    
    if i == 0:
        out = r.data
        print(out)
    else:
        out = torch.cat((r.data, out), 0)


 1.1260e-04  1.7066e-05  9.9987e-01
 9.6864e-01  2.9550e-02  1.8106e-03
 1.6505e-03  9.9835e-01  2.3984e-07
 7.2757e-01  2.4154e-01  3.0884e-02
 4.0653e-01  4.7224e-03  5.8875e-01
 3.1539e-01  6.8444e-01  1.7175e-04
 9.9681e-01  2.6299e-03  5.6410e-04
 2.1186e-03  1.7707e-03  9.9611e-01
 9.9752e-01  2.4773e-03  5.4586e-07
 9.9426e-01  5.7345e-03  3.6738e-06
 7.8239e-03  4.2739e-02  9.4944e-01
 8.8632e-05  9.9991e-01  2.4145e-08
 2.2033e-02  9.7761e-01  3.5177e-04
 3.1947e-06  1.0000e+00  3.5710e-08
 3.3151e-01  2.6989e-03  6.6579e-01
 5.8371e-06  5.5205e-07  9.9999e-01
[torch.FloatTensor of size 16x3]



In [13]:
rr = pd.DataFrame(np.array(out), columns=['EAP', 'HPL', 'MWS'], index=test.id)

In [14]:
rr.to_csv("444.csv")